In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
ensure_pyterrier_is_loaded()

import pyterrier as pt
import os
import sys
import pandas as pd
sys.path.append('..')
from chatgpt_query_reduction import ChatGPTQueryReduction


index = pt.IndexRef.of(os.path.abspath('../../pyterrier-index/index/data.properties'))

Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
chat_gpt_query_reduction = ChatGPTQueryReduction('../query-variants-in-progress', '4').as_transformer()

pipeline = chat_gpt_query_reduction >> bm25

In [3]:
for d in ['tot-test', 'tot-train', 'tot-dev']:
    !mkdir -p {d}
    queries = pt.io.read_topics(f'../../{d}/queries.xml', 'trecxml')
    processed_queries = chat_gpt_query_reduction(queries)
    processed_queries.to_json(f'{d}/queries.jsonl', lines=True, orient='records')
    run = pipeline(queries)
    persist_and_normalize_run(run, 'webis-red-04', f'{d}/run.txt')


### Final test to verify that the scores are identical to the hyperparameter tuning.

In [4]:
def report_effectiveness(dataset):
    queries = pt.io.read_topics(f'../../{dataset}/queries.xml', 'trecxml')
    run = pt.Transformer.from_df(pt.io.read_results(f'{dataset}/run.txt'))

    ret = pt.Experiment([run], queries, pt.io.read_qrels(f'../../{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10', 'recall_100', 'recall_1000'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10', 'recall_100', 'recall_1000']}

def report_pipeline():
    ret = {}
    for dataset in ['tot-train', 'tot-dev']:
        for k,v in report_effectiveness(dataset).items():
            ret[dataset.split('-')[1] + '_' + k] = v
    return pd.DataFrame([ret])

report_pipeline()

,train_ndcg_cut_10,train_recip_rank,train_P_10,train_recall_100,train_recall_1000,dev_ndcg_cut_10,dev_recip_rank,dev_P_10,dev_recall_100,dev_recall_1000
0,0.095803,0.087807,0.014,0.333333,0.54,0.107782,0.104781,0.014,0.286667,0.52
